In [1]:
from datetime import datetime
from IPython.display import Markdown as md
from IPython.display import display_markdown
import pandas as pd

todays_date = str(datetime.now().date())
month = datetime.now().strftime("%B")
year = str(datetime.now().year)

import sys
import os
sys.path.append(os.path.abspath('../utils/'))
from shared_functions import get_year_and_month, get_month_name

year, month = get_year_and_month()
month_name = get_month_name(month)

# Create start and end dates
start_date = pd.to_datetime(f'{year}-{month}-01')
end_date = pd.to_datetime(f'{year}-{month + 1}-01') - pd.Timedelta(days=1)  # Last day of the month
last_year = pd.to_datetime(f'{year-1}-{month + 1}-01') - pd.Timedelta(days=1) # for 1 year of data
date_123 = end_date - pd.Timedelta(days=123) # For 123 days of data
# display_markdown('''# NBS monthly report for {month} {year}''')
# md("# NBS monthly report for {} {}".format(month, year))

# Quick summary

The table below shows a short overview of the NBS performance operation during the last 30 days. The number of products are compared against CDSE. All columns represents the number of products in each portal except the last 3 columns. Those 3 columns represents the data flow from MET Norway to users through the portals where Volumes are measured in Tb. 


In [2]:
import pathlib
import datetime as dt
import pandas as pd
import copy
from myst_nb import glue

logsdir = pathlib.Path('../data')
hubs = ['colhub_global', 'cdse', 'esahub_global', 'colhub_AOI']
products = ['S1', 'S2L1C', 'S2L2A', 'S3', 'S5p']

data_FE = None
for h in hubs:
    csvfile = logsdir / f'products_in_{h}.csv'
    data_tmp = pd.read_csv(csvfile, header=None, names=['product', 'area', 'sensing_date', f'{h}'], parse_dates=['sensing_date'])
    # If several sensing date exist, keep the most recent one
    data_clean = copy.deepcopy(data_tmp.drop_duplicates(subset=['sensing_date', 'product', 'area'], keep='last'))
    if h == 'colhub_AOI':
        data_clean['area'] = 'colhub_aoi'
    if data_FE is None:
        data_FE = data_clean
    else:
        data_FE = data_FE.merge(data_clean, on=['sensing_date', 'product', 'area'], how='outer')

data = data_FE
data.set_index('sensing_date', inplace=True)
data.sort_index(inplace=True)
data.fillna(0, inplace=True)

data = data[
    (data.index >= start_date) & 
    (data.index <= end_date) & 
    (data['area'] == 'colhub_aoi')
]

data_s1 = data[
    (data['product'] == 'S1')
][['colhub_global', 'cdse', 'colhub_AOI']].copy()
col_s1 = data_s1['colhub_global'].sum()
arc_s1 = data_s1['colhub_AOI'].sum()
cdse_s1 = data_s1['cdse'].sum()

data_s2l1 = data[
    (data['product'] == 'S2L1C')
][['colhub_global', 'cdse', 'colhub_AOI']].copy()
col_s2l1 = data_s2l1['colhub_global'].sum()
arc_s2l1 = data_s2l1['colhub_AOI'].sum()
cdse_s2l1 = data_s2l1['cdse'].sum()

data_s2l2 = data[
    (data['product'] == 'S2L2A')
][['colhub_global', 'cdse', 'colhub_AOI']].copy()
col_s2l2 = data_s2l2['colhub_global'].sum()
arc_s2l2 = data_s2l2['colhub_AOI'].sum()
cdse_s2l2 = data_s2l2['cdse'].sum()

data_s3 = data[
    (data['product'] == 'S3')
][['colhub_global', 'cdse', 'colhub_AOI']].copy()
col_s3 = data_s3['colhub_global'].sum()
arc_s3 = data_s3['colhub_AOI'].sum()
cdse_s3 = data_s3['cdse'].sum()

data_s5 = data[
    (data['product'] == 'S5p')
][['colhub_global', 'cdse', 'colhub_AOI']].copy()
col_s5 = data_s5['colhub_global'].sum()
arc_s5 = data_s5['colhub_AOI'].sum()
cdse_s5 = data_s5['cdse'].sum()

In [3]:
def get_product_type(product):
    if product[0:2] == 'S1':
        type = product.split('_')[2]
    elif product[0:2] == 'S2':
        type = product.split('_')[1]
        if not type.startswith('M'):
            type = 'Unknown'
    elif product[0:2] == 'S3':
        tmp = product.split('_')
        if tmp[1] == 'SL':
            type = 'SLSTR_L' + tmp[2]
        elif tmp[1] == 'SR':
            type = 'SRAL_L' + tmp[2]
        elif tmp[1] == 'OL':
            type = 'OLCI_L' + tmp[2]
        else:
            type = 'Unknown'
    else:
        type = 'Unknown'
    if 'DTERRENG' in product:
        type = type + '_DTERRENG'
    return type

def get_data(file):
    data = pd.read_csv(file, header=None, names=['download_time', 'user', 'product', 'product_type', 'size', 'download_duration', 'not_avail']\
                        , parse_dates=['download_time'], index_col='download_time')
    data['satellite'] = data['product'].apply(lambda x: x[0:2])
    data['product_type'] = data['product'].apply(get_product_type)
    return data[data['product_type'] != 'Unknown']

csvfile = logsdir / 'NBS_frontend-global_outputs.csv'
nbs_global = get_data(csvfile)
nbs_global.sort_index(inplace=True)
nbs_global.fillna(0, inplace=True)
nbs_30 = nbs_global[
    (nbs_global.index >= start_date) & 
    (nbs_global.index <= end_date)
]

col_use = len(nbs_30['user'].unique())
col_nb = nbs_30.count()['product_type']
col_vol = nbs_30.sum()['size']/1024/1024/1024/1024 #in Tb

csvfile = logsdir / 'NBS_frontend-AOI_outputs.csv'
nbs_AOI = get_data(csvfile)
nbs_AOI.sort_index(inplace=True)
nbs_AOI.fillna(0, inplace=True)
aoi_30 = nbs_AOI[
    (nbs_AOI.index >= start_date) & 
    (nbs_AOI.index <= end_date)
]

arc_use = len(aoi_30['user'].unique())
arc_nb = aoi_30.count()['product_type']
arc_vol = aoi_30.sum()['size']/1024/1024/1024/1024 #in Tb

In [4]:
import pathlib
import datetime as dt
import pandas as pd
import copy
from myst_nb import glue

logsdir = pathlib.Path('../data')

def read_dhus_logs_details(file):
    data = pd.read_csv(file, header=None, names=['day', 'product_type', 'action', 'volume', 'number', 'timeliness'], 
                        parse_dates=['day'], index_col=['day'])
    return data

def read_csv(file):
    data_tmp = pd.read_csv(file, header=None, names=['product_type', 'day', 'number', 'volume'], parse_dates=['day'], sep=';')
    # If several sensing date exist, keep the numbers only for the most recent script run (ie highest day index)
    data_clean = copy.deepcopy(data_tmp.drop_duplicates(subset=['product_type', 'day'], keep='last')).set_index('day').sort_index()
    data_clean['volume'] = data_clean['volume'] / 1024. / 1024
    return data_clean

data_s1 = read_dhus_logs_details(logsdir / 'S1-backend-AOI_inputs.csv')
data_s2l1c = read_dhus_logs_details(logsdir / 'S2L1C-backend-AOI_inputs.csv')
data_s2l2a = read_dhus_logs_details(logsdir / 'S2L2A-backend-AOI_inputs.csv')
data_s3 = read_dhus_logs_details(logsdir / 'S3-backend-AOI_inputs.csv')
data_s5 = read_dhus_logs_details(logsdir / 'S5p-backend-AOI_inputs.csv')

all_colhub = pd.concat([data_s1, data_s2l1c, data_s2l2a, data_s3, data_s5])
all_colhub = all_colhub[
    (all_colhub.index <= end_date)
]

csvfile = pathlib.Path('../data/nb_products_volume_per_sensing_date.csv')
all_netcdf = read_csv(csvfile)
all_netcdf = all_netcdf[
    (all_netcdf.index <= end_date)
]

alle = pd.concat([all_colhub, all_netcdf])

all_vol_be = alle['volume'].sum() / 1024  # in Tb

lately = alle[
    (alle.index >= start_date) & 
    (alle.index <= end_date)
]
vol_be = lately['volume'].sum() / 1024  # in Tb

#nb_be = be_s1 + be_s2l1 + be_s2l2 + be_s3 + be_s5

In [6]:
glue("col_s1", col_s1)
glue("col_s2l1", col_s2l1)
glue("col_s2l2", col_s2l2)
glue("col_s3", col_s3)
glue("col_s5", col_s5)
glue("arc_s1", arc_s1)
glue("arc_s2l1", arc_s2l1)
glue("arc_s2l2", arc_s2l2)
glue("arc_s3", arc_s3)
glue("arc_s5", arc_s5)
glue("col_use", col_use)
glue("col_nb", col_nb)
glue("col_vol", col_vol)
glue("arc_use", arc_use)
glue("arc_nb", arc_nb)
glue("arc_vol", arc_vol)
glue("cdse_s1", cdse_s1)
glue("cdse_s2l1", cdse_s2l1)
glue("cdse_s2l2", cdse_s2l2)
glue("cdse_s3", cdse_s3)
glue("cdse_s5", cdse_s5)

8694.0

40447.0

40625.0

36265.0

15860.0

...

11793.0

40776.0

40750.0

54947.0

0.0

| Portals | S1 | S2L1C | S2L2A | S3 | S5p | Nb of users | Nb of products | Volume |
|:---------:|:---------:|:--------:|:---------:|:----------:|:----------:|:-------------:|:---------:|:----------:|
| colhub.met.no | {glue:text}`col_s1:.0f` | {glue:text}`col_s2l1:.0f` | {glue:text}`col_s2l2:.0f` | {glue:text}`col_s3:.0f` | {glue:text}`col_s5:.0f` | {glue:text}`col_use` | {glue:text}`col_nb` | {glue:text}`col_vol:.3f` |
| colhub-archive.met.no | {glue:text}`arc_s1:.0f` | {glue:text}`arc_s2l1:.0f` | {glue:text}`arc_s2l2:.0f` | {glue:text}`arc_s3:.0f` | {glue:text}`arc_s5:.0f` | {glue:text}`arc_use` | {glue:text}`arc_nb` | {glue:text}`arc_vol:.3f` |
| dataspace.copernicus.eu | {glue:text}`cdse_s1:.0f` | {glue:text}`cdse_s2l1:.0f` | {glue:text}`cdse_s2l2:.0f` | {glue:text}`cdse_s3:.0f` | {glue:text}`cdse_s5:.0f` |  | | |


In [7]:
md("Finally, the total amount of disk space dedicated to the NBS project, including either products in SAFE and NetCDF formats, represents {} Tb.".format(int(all_vol_be)))

Finally, the total amount of disk space dedicated to the NBS project, including either products in SAFE and NetCDF formats, represents 5845 Tb.

In [8]:
# Monthly volumes (in Tb)
lately = alle[
    (alle.index >= last_year) & 
    (alle.index <= end_date)
]
year = (lately['volume'].sum()/1024.)
after_year = year + all_vol_be
short = alle[
    (alle.index >= date_123) & 
    (alle.index <= end_date)
]
m6 = (short['volume'].sum()/1024.)
after_6m = m6 + all_vol_be

In [9]:
md("Due to tracking the data ingested and produced for the NBS project in the last year it is possible to forecast the upcoming need for disk space. As long as data flows follows the same pattern than last year, in 6 months the total disk space will grow until {} Tb; while in 12 months it is forecast to become {} Tb.".format(int(after_6m), int(after_year)))

Due to tracking the data ingested and produced for the NBS project in the last year it is possible to forecast the upcoming need for disk space. As long as data flows follows the same pattern than last year, in 6 months the total disk space will grow until 6206 Tb; while in 12 months it is forecast to become 6870 Tb.